In [2]:
from src.transformers import BertGenerationTokenizer, BertGenerationDecoder, BertGenerationConfig,  BertGenerationEncoder

/home/ygambhir/anaconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Dataset already exists at /home/ygambhir/.convokit/downloads/wikipedia-politeness-corpus
1000/4353 utterances processed
2000/4353 utterances processed
3000/4353 utterances processed
4000/4353 utterances processed
4353/4353 utterances processed
train size = 1743, test size = 435
Initialized default classification model (standard scaled logistic regression).


In [4]:
import torch

In [ ]:
### This is an original experiment with 'BERTGeneration', but it seems like this isn't the model we should use

tokenizer = BertGenerationTokenizer.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder')
config = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config_enc = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config.is_decoder = True
model = BertGenerationDecoder.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder', config=config)
enc_model = BertGenerationEncoder.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder', config=config_enc)

In [ ]:
### Further exploration of BERTGeneration
inputs = tokenizer(["Hello, my dog is cute"], return_tensors="pt")
label_outputs = tokenizer(["and stuff and winaokwnr ajdns"], return_tensors="pt")
print(inputs['input_ids'].shape)
print(label_outputs['input_ids'].shape)
outputs = model(inputs['input_ids'], inputs['attention_mask'], labels=label_outputs['input_ids'])
print(outputs[0])

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0) 

In [ ]:
outputs = model(**input_ids)

In [6]:
from src.transformers import EncoderDecoderModel, BertTokenizer

In [22]:
## This is the Seq2Seq model we want to use for our task, more details here:
## https://huggingface.co/transformers/model_doc/encoderdecoder.html

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert from pre-trained checkpoints


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
input_ids = torch.tensor(tokenizer.encode(["What is the", "and what else"], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
label_outputs = torch.tensor(tokenizer.encode(["name of the gym?", "can we do here"], add_special_tokens=True)).unsqueeze(0)
#outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=label_outputs)
#print(outputs[-1].shape)
print(input_ids.shape)

torch.Size([1, 4])


In [1]:
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id, do_sample=True)
print(tokenizer.decode(generated[0], skip_special_tokens=True))

NameError: name 'model' is not defined

In [ ]:
from eecs598.transformers.src.transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
## Simple training script with just a couple examples to demo
## Based on https://huggingface.co/transformers/training.html#pytorch

num_epochs = 10
text_batch = ["I love Pixar.", "I don't care for Pixar."]
input_ids = torch.tensor(tokenizer.encode(text_batch, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
label_outputs = torch.tensor(tokenizer.encode(["so much","at all"], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
for i in range(num_epochs):
    print(i)
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=label_outputs)
    loss = outputs[0]
    loss.backward()
    optimizer.step()